In [81]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_excel('eines.xlsx')
data.head(10)

,_time,SkidNr,Line,Camera,Recognized,Quality1,Quality2,Quality3,Quality4,Quality_Min
0,2022-06-27T14:30:04.000+0200,1322,3,6,OK,0.88,0.93,0.91,0.92,0.88
1,2022-06-27T14:29:58.000+0200,365,3,4,OK,0.97,0.93,0.97,0.89,0.89
2,2022-06-27T14:29:30.000+0200,329,2,4,OK,0.95,0.91,0.94,0.96,0.91
3,2022-06-27T14:29:27.000+0200,104,1,2,OK,0.96,0.84,0.96,0.93,0.84
4,2022-06-27T14:29:23.000+0200,576,2,6,OK,0.98,0.91,0.94,0.95,0.91
5,2022-06-27T14:29:02.000+0200,427,2,1,OK,0.91,0.88,0.87,0.85,0.85
6,2022-06-27T14:28:55.000+0200,506,1,3,OK,0.95,0.90,0.96,0.96,0.90
7,2022-06-27T14:28:47.000+0200,766,2,2,OK,0.95,0.91,0.93,0.93,0.91
8,2022-06-27T14:28:43.000+0200,1465,1,5,OK,0.86,0.92,0.95,0.90,0.86
9,2022-06-27T14:28:33.000+0200,189,3,2,OK,0.94,0.83,0.93,0.93,0.83


In [111]:
#parse column _time to datetime format
data['_time'] = pd.to_datetime(data['_time'])

data['Date'] = pd.to_datetime(data['_time']).dt.date
data['Time'] = pd.to_datetime(data['_time']).dt.time
second_column = data.pop('Date')
data.insert(1, 'Date', second_column)
third_column = data.pop('Time')
data.insert(2, 'Time', third_column)
data.pop('_time')



KeyError: ignored

In [110]:
#bin encoding of column data['Recognized']
#OK - 1, NOK - 0
#data['Recognized'] = data['Recognized'].map(lambda x: 1 if x=='OK' else 0)
#data.head(10)

0         0
1         0
2         0
3         0
4         0
         ..
211258    0
211259    0
211260    0
211261    0
211262    0
Name: Recognized, Length: 211263, dtype: int64


In [126]:
data = data[data['Recognized'] == 'OK']


(0, 11)

In [127]:
oe = preprocessing.OrdinalEncoder()
#data['Recognized'] = oe.fit_transform(data['Recognized'])
data['Recognized'].unique()

array([], dtype=int64)

In [ ]:
# splitting data: 80% training and 20% test
features = list(data.columns.values.tolist())
X = data.loc[:, features]
y = data.loc[:, ['Recognized']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = 0.2)

In [ ]:
#Isolation Forest model
model=IsolationForest(n_estimators=100, max_samples='auto', contamination=float(0.1), max_features=1.0)
model.fit(data[['Quality1']])

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  "X does not have valid feature names, but"


IsolationForest(contamination=0.1)

In [97]:
data.isnull().sum()
data.groupby(['Recognized']).size()

Recognized
0    211263
dtype: int64